**BEAST2 v2.1.0: Bayesian Evolutionary Anaysis by Sampling Trees**
* Adapted from python_cipres_quickstart notebook, last updated 02/02/2017

This notebook is designed to allow you to run your own data through the CIPRES REST API (CRA) BEAST2 interface: the BEAST2 v2.1.0 tool (tool ID: beast2_xsede). 

** TO DO before running code in this notebook: **

1) Make an account (if you have one, skip this step)
 
 - Register for a CIPRES REST API account at https://www.phylo.org/restusers/login.action. 

2) Create an application (if you have one, skip this step)
    
 - ** An application is an easy way to name, organize, and keep track of your processes.**
 - Once you have logged into your account, go to Developer > Application Management (https://www.phylo.org/restusers/developer). Create a new application by following the instructions listed. 

3) Remember that the input file format for BEAST 2 is a properly formatted BEAST XML file 
- This input file can be created using the tool BEAUti.
- BEAST/BEAUTi manual: http://www.molecularevolution.org/molevolfiles/beast/BEAST14_MANUAL-7-6-07.pdf).

The following lines import/install necessary packages.

In [ ]:
# provides support for both Python 2 and 3
from __future__ import print_function

In [ ]:
# install the CIPRES REST API client library 
!pip install python_cipres

In [ ]:
import python_cipres.client as CipresClient
import requests
import os

**BEFORE RUNNING NEXT LINE**
<br>Look at your Application Information.</br>
Replace the following fields with your own information:
 - **<span style="color:red">NAME_OF_YOUR_APPLICATION</span>** is the name of your application
 - **<span style="color:red">YOUR_APPLICATION_ID</span>** is the Application ID listed under your app

This should look similar to this example:

- **<span style="color:purple">appname = "demo"</span>**
- **<span style="color:purple">appid = "mydemo-8126CB900A964FA1AD14174512F9403C"</span>**

In [ ]:
url = "https://cipresrest.sdsc.edu/cipresrest/v1"
appname = "NAME_OF_YOUR_APPLICATION"
appid = "YOUR_APPLICATION_ID"

**BEFORE RUNNING NEXT LINE**
<br>Replace the following fields with your own information:</br>
 - **<span style="color:red">REPLACE_WITH_YOUR_USERNAME</span>** with your CRA account username
 - **<span style="color:red">REPACE_WITH_YOUR_PASSWORD</span>** with your CRA account password

In [ ]:
username = "REPLACE_WITH_YOUR_USERNAME"
password = "REPLACE_WITH_YOUR_PASSWORD"

Now we create a CipresClient to communicate with the CIPRES REST API:

In [ ]:
client = CipresClient.Client(appname, appid, username, password, url)

**BEFORE RUNNING NEXT LINE**
<br>Set the variable 'inputFilename' to the path of your input XML file from your home directory.

In [ ]:
inputFilename = './PATH_TO_YOUR_FILE/YOUR_FILE_NAME'

The following code submits the job to CIPRES with the file you specified above. 

This is a basic submission. 
The parameters mean the following:
- **<span style="color:purple"> 'tool' : 'BEAST2_XSEDE'</span>**
  - selects BEAST2_XSEDE as the CIPRES tool you want to run
- **<span style="color:purple"> 'nu_ _patterns_ _' : '150'</span>**
  - specifies the number of patterns in XML file (can be found in file - ex. 150)
- **<span style="color:purple">'input.infile_' : 'inputFilename'</span>**
  - specifies the file to run (the file you specified earlier)
- **<span style="color:purple">'metadata.statusEmail' : 'true'</span>**
  - an email will be sent to user when job has finished
     
**If you would like to add parameters, please reference: **

1) the BEAST2 tool info page: http://www.phylo.org/index.php/rest/beast2_xsede.html
- This page lists all the possible parameters you can run.

2) the tool configuration helper, selecting BEAST2 as your tool:
    https://www.phylo.org/restusers/docs/cipresXml
- This is an easy-to-use tool that generates a command based on the parameter specifications you wish to apply to your data in BEAST2. 
- **Using the tool configuration helper:** Fill out the Simple/Advanced Parameters fields you are interested in, then press 'View'. This will give you a list of parameters. You can copy each of the parameters headed by 'vparam.' inside the curly braces after the 'tool' parameter (do not include the 'vparam.' header). Example:
 
         job = client.submitJob(
                 {'tool' : 'BEAST2_XSEDE', 
                  'nu_patterns_' : '150', 
                  'overwrite_logs_' : '1', 
                  'runtime_' : '1.0'}
                 {'input.infile_': inputFilename},
                 {'metadata.statusEmail': 'true'});
                 
Please note that each parameter from the tool configuration helper is listed in the following format:
- **<span style="color:purple">'vparam.parameter_name' = 'parameter_value'</span>**

You must convert the format of each parameter to the following before adding it to your submitJob function:</br>

- **<span style="color:purple">'parameter_name' : 'parameter_value'</span></br>**

In [ ]:
job = client.submitJob(
        {'tool' : 'BEAST2_XSEDE', 'nu_patterns_' : '150'},
        {'input.infile_': inputFilename},
        {'metadata.statusEmail': 'true'});

Show job status information:

In [ ]:
job.show(messages="true")

The job was submitted successfully if no errors are shown. To update (refresh) job status:

In [ ]:
job.update()
job.show(messages="true")

The code below ensures the program will wait for your job to finish before proceeding. 
<br>**Wait for the "Job ... is finished" output to appear before running anything else:**</br>

In [ ]:
job.waitForCompletion()
print("Job %s finished.  isError() returns %s" % (job.jobHandle, job.isError()))

To download your result files to a new directory (called BEAST2_Results here):

In [ ]:
downloadDir = "./BEAST2_Results"
try:
    os.mkdir(downloadDir)
except:
    pass 
job.downloadResults(downloadDir)

Once you've downloaded a job's results you should delete the job to conserve space on CIPRES.  If you delete a job that is still running or is queued to run, it will be cancelled.

In [ ]:
job.delete()